In [17]:
#!/usr/bin/python

import os
import pickle
import re
import sys

sys.path.append("/Users/karthikr/Documents/enron/code/ud120-projects/tools/")
from parse_out_email_text import parseOutText

"""
    Starter code to process the emails from Sara and Chris to extract
    the features and get the documents ready for classification.

    The list of all the emails from Sara are in the from_sara list
    likewise for emails from Chris (from_chris)

    The actual documents are in the Enron email dataset, which
    you downloaded/unpacked in Part 0 of the first mini-project. If you have
    not obtained the Enron email corpus, run startup.py in the tools folder.

    The data is stored in lists and packed away in pickle files at the end.
"""


from_sara  = open("/Users/karthikr/Documents/enron/code/ud120-projects/text_learning/from_sara.txt", "r")
from_chris = open("/Users/karthikr/Documents/enron/code/ud120-projects/text_learning/from_chris.txt", "r")

from_data = []
word_data = []

### temp_counter is a way to speed up the development--there are
### thousands of emails from Sara and Chris, so running over all of them
### can take a long time
### temp_counter helps you only look at the first 200 emails in the list so you
### can iterate your modifications quicker
temp_counter = 0


for name, from_person in [("sara", from_sara), ("chris", from_chris)]:
    for path in from_person:
        ### only look at first 200 emails when developing
        ### once everything is working, remove this line to run over full dataset
        temp_counter += 1
        if temp_counter < 1000:
            path = os.path.join('/Users/karthikr/Documents/enron/', path[:-1])
            print path
            email = open(path, "r")

            ### use parseOutText to extract the text from the opened email
            parsed_email_text = parseOutText(email)
            ### use str.replace() to remove any instances of the words
            ### ["sara", "shackleton", "chris", "germani"]
            parsed_email_text.replace('sara', '')
            parsed_email_text.replace('shackleton', '')
            parsed_email_text.replace('chris', '')
            parsed_email_text.replace('germani', '')

            ### append the text to word_data
            word_data.append(parsed_email_text)
            ### append a 0 to from_data if email is from Sara, and 1 if email is from Chris
            if name == "sara":
                from_data.append(0)
            elif name == "chris":
                from_data.append(1)

            email.close()

print "emails processed"
from_sara.close()
from_chris.close()

pickle.dump( word_data, open("/Users/karthikr/Documents/enron/code/ud120-projects/text_learning/your_word_data.pkl", "w") )
pickle.dump( from_data, open("/Users/karthikr/Documents/enron/code/ud120-projects/text_learning/your_email_authors.pkl", "w") )





### in Part 4, do TfIdf vectorization here




/Users/karthikr/Documents/enron/maildir/bailey-s/deleted_items/101.
/Users/karthikr/Documents/enron/maildir/bailey-s/deleted_items/106.
/Users/karthikr/Documents/enron/maildir/bailey-s/deleted_items/132.
/Users/karthikr/Documents/enron/maildir/bailey-s/deleted_items/185.
/Users/karthikr/Documents/enron/maildir/bailey-s/deleted_items/186.
/Users/karthikr/Documents/enron/maildir/bailey-s/deleted_items/187.
/Users/karthikr/Documents/enron/maildir/bailey-s/deleted_items/193.
/Users/karthikr/Documents/enron/maildir/bailey-s/deleted_items/195.
/Users/karthikr/Documents/enron/maildir/bailey-s/deleted_items/214.
/Users/karthikr/Documents/enron/maildir/bailey-s/deleted_items/215.
/Users/karthikr/Documents/enron/maildir/bailey-s/deleted_items/233.
/Users/karthikr/Documents/enron/maildir/bailey-s/deleted_items/242.
/Users/karthikr/Documents/enron/maildir/bailey-s/deleted_items/243.
/Users/karthikr/Documents/enron/maildir/bailey-s/deleted_items/244.
/Users/karthikr/Documents/enron/maildir/bailey-s

/Users/karthikr/Documents/enron/maildir/shackleton-s/all_documents/1049.
/Users/karthikr/Documents/enron/maildir/shackleton-s/all_documents/10491.
/Users/karthikr/Documents/enron/maildir/shackleton-s/all_documents/10499.
/Users/karthikr/Documents/enron/maildir/shackleton-s/all_documents/10504.
/Users/karthikr/Documents/enron/maildir/shackleton-s/all_documents/10506.
/Users/karthikr/Documents/enron/maildir/shackleton-s/all_documents/10509.
/Users/karthikr/Documents/enron/maildir/shackleton-s/all_documents/1051.
/Users/karthikr/Documents/enron/maildir/shackleton-s/all_documents/10510.
/Users/karthikr/Documents/enron/maildir/shackleton-s/all_documents/10511.
/Users/karthikr/Documents/enron/maildir/shackleton-s/all_documents/10513.
/Users/karthikr/Documents/enron/maildir/shackleton-s/all_documents/10514.
/Users/karthikr/Documents/enron/maildir/shackleton-s/all_documents/10521.
/Users/karthikr/Documents/enron/maildir/shackleton-s/all_documents/10522.
/Users/karthikr/Documents/enron/maildir/

In [21]:
#!/usr/bin/python

import pickle
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
numpy.random.seed(42)


### The words (features) and authors (labels), already largely processed.
### These files should have been created from the previous (Lesson 10)
### mini-project.
words_file = "/Users/karthikr/Documents/enron/code/ud120-projects/text_learning/your_word_data.pkl" 
authors_file = "/Users/karthikr/Documents/enron/code/ud120-projects/text_learning/your_email_authors.pkl"
word_data = pickle.load( open(words_file, "r"))
authors = pickle.load( open(authors_file, "r") )



### test_size is the percentage of events assigned to the test set (the
### remainder go into training)
### feature matrices changed to dense representations for compatibility with
### classifier functions in versions 0.15.2 and earlier
from sklearn import cross_validation
features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                             stop_words='english')
features_train = vectorizer.fit_transform(features_train)
features_test  = vectorizer.transform(features_test)



### a classic way to overfit is to use a small number
### of data points and a large number of features;
### train on only 150 events to put ourselves in this regime
features_train = features_train[:150]
labels_train   = labels_train[:150]
#print features_test
model = DecisionTreeClassifier()
fit_model = model.fit(features_train, labels_train)

pred = fit_model.predict(features_test)

#print fit_model.feature_importances_
fi = np.array(fit_model.feature_importances_)
for v in fi:
    if v > 0.01:
        print v
print np.max(fi)
### your code goes here





0.0
